In [1]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from numpy import genfromtxt
my_data = genfromtxt('my_data.csv', delimiter=',')

In [2]:
X = my_data[:, 1:-1]  # Tomamos las columnas de significancia estadística
y = my_data[:, -1]   #Tomamos la columna con las etiquetas

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#Estandarizamos los datos con los que trabajaremos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
#Instanciamos el Perceptron multicapa, en esta caso decidí usar el método Adam como mi optimizador
mlp = MLPClassifier(hidden_layer_sizes=(50), max_iter=1000000, solver="adam",random_state=42)
#Entrenamos el Clasificador
mlp.fit(X_train, y_train)
#Predecimos las salidas del conjunto de testeo
y_pred = mlp.predict(X_test)
#Evaluamos el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')

Accuracy: 100.00%


In [4]:
y_test

array([2., 2., 1., 2., 1., 2., 2., 1., 2., 3., 1., 2., 1., 1., 3., 3., 3.,
       1., 3., 3., 2., 2., 1., 2.])

In [5]:
y_pred

array([2., 2., 1., 2., 1., 2., 2., 1., 2., 3., 1., 2., 1., 1., 3., 3., 3.,
       1., 3., 3., 2., 2., 1., 2.])

In [6]:
import plotly.graph_objects as go
#Buscamos los indices de los puntos bien y mal clasificados
indices_puntos_bien_clasificados = [i for i in range(len(y_test)) if y_test[i] == y_pred[i]]
indices_puntos_mal_clasificados = [i for i in range(len(y_test)) if y_test[i] != y_pred[i]]
#Como menciona la práctica tomamos unicamente 2 características aunque algunas referencias establecen que es mas apropiado usar un PCA
#Guardamos los datos de las plantas correctamente clasificadas por el modelo
x_correctas = [X_test[i][0] for i in indices_puntos_bien_clasificados]
y_correctas = [X_test[i][1] for i in indices_puntos_bien_clasificados]
#Guardamos los de los datos erroneamente clasificados
x_incorrectas = [X_test[i][0] for i in indices_puntos_mal_clasificados]
y_incorrectas = [X_test[i][1] for i in indices_puntos_mal_clasificados]
#Instanciamos el objeto Figure
fig = go.Figure()
# Agregamos los puntos clasificados adecuadamente a la gráfica con un marcador según la clase a la que pertenecen
for label_type in set(y_test):
    indices = [i for i, etiqueta in enumerate(y_test) if etiqueta == label_type]
    x = [X_test[i][0] for i in indices]
    y = [X_test[i][1] for i in indices]
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='markers',
        marker=dict(size=6),
        name=f'etiqueta {label_type}'
    ))

#Agregamos los puntos erroneamente clasificados con un hueco
fig.add_trace(go.Scatter(
    x=x_incorrectas,
    y=y_incorrectas,
    mode='markers',
    marker=dict(size=10, symbol='circle', line=dict(color='black', width=2),color='white'),
    name='Puntos mal clasificados'
))
fig.update_layout(
    title='Gráfica de los datos de prueba de clasificación del dataset Iris',
    xaxis=dict(title='Característica 1'),
    yaxis=dict(title='Característica 2')
)
#Mostramos la figura
fig.show()